In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [ ]:
# https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [ ]:
df = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')

In [ ]:
df.head()

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2022.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [ ]:
len(df.columns)

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 41.45
* 46.45
* 51.45
* 56.45

In [ ]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

In [ ]:
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
df['duration'].describe()

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [ ]:
less_than_hour_df = df[(df.duration >= 1) & (df.duration <= 60)]

In [ ]:
len(less_than_hour_df) / float(len(df))

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [ ]:
categorical = ['PULocationID', 'DOLocationID']

two_features = less_than_hour_df[categorical].astype(str)

In [ ]:
two_features_dict_version = two_features.to_dict(orient='records')

In [ ]:
dv = DictVectorizer()
X_train = dv.fit_transform(two_features_dict_version)

In [ ]:
X_train.shape

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 6.99
* 11.99
* 16.99
* 21.99

In [ ]:
target = 'duration'
y_train = less_than_hour_df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2022). 

What's the RMSE on validation?

* 7.79
* 12.79
* 17.79
* 22.79

In [ ]:
def read_dataframe(filename):
    if filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')

In [ ]:
prediction_dict = df_val[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [ ]:
target = 'duration'
X_val = dv.transform(prediction_dict)
y_val = df_val[target].values

In [ ]:
X_val.shape

In [ ]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)